## RSI

RSI指標(英文:Relative Strength Index , RSI)又稱為 相對強弱指標，用來評估「買賣盤雙方力道的強弱」，由Wells Wilder於1978年發明。

> 「當價格上升時，市場情緒也會隨之上漲，此時RSI指標上升，當價格下降時，市場情緒也會隨之下降，此時RSI指標下降。」

```
RSI (相對強弱指標) = n日漲幅平均值÷(n日漲幅平均值+ n日跌幅平均值) × 100
其中，
n日漲幅平均值 = n日內上漲日總上漲幅度加總 / n
n日跌幅平均值 = n日內下跌日總下跌幅度加總 / n

n 一般取14，例如最近14天中，有10天是上漲，累積漲幅加總為 5%，平均漲幅就是 5% / 14 = 0.35%
有4天是下跌，累積跌幅是 -7%，平均跌幅就是 7% / 14 = 0.5%
RSI = 0.35% / (0.35% + 0.5%) * 100 = 41.17
```

> RSI指標的數值介於0到100之間，在一般大眾的通常定義之中，當RSI指標為70以上時，被認為是超買區，也就是市場情緒火熱到了某種程度，此時買方的比例太多，已經超出了正常的買方情緒，而若是RSI指標低於30時，則被認為是超賣區，這表示當下市場賣方的比例太多，賣方情緒超出正常範圍。

In [1]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover #, TrailingStrategy
# from backtesting.test import SMA

import pandas as pd
import pandas_ta as ta
import yfinance as yf

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df = yf.download("^SPX", period="60d", interval="5m")

[*********************100%%**********************]  1 of 1 completed


## RSI趨勢交叉策略

> 當RSI超過50或是跌破50，並且商品價格本身同時站上突破或跌破均線時，可能意味市場轉折，即將發生趨勢，這可能會是一個交易的機會，交易者可以考慮價格向上交叉出現趨勢，且RSI突破50時且持續增加時買進，或是價格向下交叉出現趨勢，且RSI跌破50時且持續減少時賣出。

In [3]:
class RsiOscillator_01(Strategy):
    
    sma_len = 17
    rsi_window = 14

    def init(self):
        close = pd.Series(self.data.Close)
        self.sma_long = self.I(ta.sma, close, 14)
        self.rsi = self.I(ta.rsi, close, self.rsi_window)

    def next(self):
        price = self.data.Close[-1]
        sma_long = self.sma_long[-1]

        if self.position:
            # 做多時，rsi 向下穿 50， 平倉
            if self.position.is_long and crossover(50, self.rsi):
                self.position.close()

            # 做空時，rsi 向上穿 50， 平倉
            if self.position.is_short and crossover(self.rsi, 50):
                self.position.close()
        else:
            # 價格低於均線，rsi 向上穿 50，做多
            if price < sma_long and crossover(self.rsi, 50):
                self.buy()

            # 價格高於均線，rsi 向下穿 50，做空
            if price > sma_long and crossover(50, self.rsi):
                self.sell()


bt = Backtest(df, RsiOscillator_01, commission=0.0002)

stats = bt.run()
print(stats)
display(stats._trades.tail())
# bt.plot()

Start                     2023-12-05 09:30...
End                       2024-03-01 10:35...
Duration                     87 days 01:05:00
Exposure Time [%]                   18.999133
Equity Final [$]                  9585.984037
Equity Peak [$]                    10008.6747
Return [%]                           -4.14016
Buy & Hold Return [%]                12.14193
Return (Ann.) [%]                  -15.192711
Volatility (Ann.) [%]                1.734163
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.440555
Avg. Drawdown [%]                   -4.440555
Max. Drawdown Duration       86 days 23:00:00
Avg. Drawdown Duration       86 days 23:00:00
# Trades                                  109
Win Rate [%]                        14.678899
Best Trade [%]                       0.404087
Worst Trade [%]                     -0.500923
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
104,1,4534,4539,5081.726103,5080.680176,-1.045927,-0.000206,2024-02-29 10:20:00-05:00,2024-02-29 10:45:00-05:00,0 days 00:25:00
105,1,4540,4541,5086.566915,5080.089844,-6.477071,-0.001273,2024-02-29 10:50:00-05:00,2024-02-29 10:55:00-05:00,0 days 00:05:00
106,1,4542,4543,5083.406615,5081.020020,-2.386595,-0.000469,2024-02-29 11:00:00-05:00,2024-02-29 11:05:00-05:00,0 days 00:05:00
107,1,4544,4548,5084.026368,5079.910156,-4.116211,-0.000810,2024-02-29 11:10:00-05:00,2024-02-29 11:30:00-05:00,0 days 00:20:00
108,1,4590,4615,5086.336888,5106.890137,20.553249,0.004041,2024-02-29 15:00:00-05:00,2024-03-01 10:35:00-05:00,0 days 19:35:00


## RSI超買超賣策略

> 當RSI指標過高或過低時，表示市場可能進入超買超賣區間，當價格超買時，表示多頭趨勢強勁，可以一起看多，當價格進入超賣空間時，表示空頭市場強勁，可以一起看空，RSI指標普遍超過70時會被視作進入超買區，低於30時則是會被視作超賣區。

In [4]:
class RsiOscillator_02(Strategy):
    
    rsi_window = 14
    upper_band = 70
    lower_band = 30

    def init(self):
        close = pd.Series(self.data.Close)
        self.rsi = self.I(ta.rsi, close, self.rsi_window)

    def next(self):
        if self.position:
            # 超賣入場做多，向上穿變成超買時平倉
            if self.position.is_long and crossover(self.rsi, self.upper_band):
                self.position.close()

            # 超買入場做空，向下穿變成超賣時離場
            if self.position.is_short and crossover(self.lower_band, self.rsi):
                self.position.close()
        else:
            # 超買
            if self.rsi[-1] > self.upper_band:
                self.sell()

            # 超賣
            if self.rsi[-1] < self.lower_band:
                self.buy()


bt = Backtest(df, RsiOscillator_02, commission=0.0002)

stats = bt.run()
print(stats)
display(stats._trades.tail())

Start                     2023-12-05 09:30...
End                       2024-03-01 10:35...
Duration                     87 days 01:05:00
Exposure Time [%]                   87.261698
Equity Final [$]                  9285.426077
Equity Peak [$]                  10131.068586
Return [%]                          -7.145739
Buy & Hold Return [%]                12.14193
Return (Ann.) [%]                  -27.110971
Volatility (Ann.) [%]                5.049463
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -8.592708
Avg. Drawdown [%]                   -0.922284
Max. Drawdown Duration       83 days 22:25:00
Avg. Drawdown Duration        7 days 21:45:00
# Trades                                   26
Win Rate [%]                        38.461538
Best Trade [%]                       0.856891
Worst Trade [%]                     -2.519793
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
21,1,3646,3745,5019.503700,4979.450195,-40.053505,-0.007980,2024-02-12 14:20:00-05:00,2024-02-14 09:35:00-05:00,1 days 19:15:00
22,-1,3746,3905,4971.095426,5007.350098,-36.254672,-0.007293,2024-02-14 09:40:00-05:00,2024-02-16 09:55:00-05:00,2 days 00:15:00
23,1,3906,4133,5007.981376,4977.770020,-30.211357,-0.006033,2024-02-16 10:00:00-05:00,2024-02-21 15:55:00-05:00,5 days 05:55:00
24,-1,4134,4339,5037.822312,5072.810059,-34.987746,-0.006945,2024-02-22 09:30:00-05:00,2024-02-26 13:35:00-05:00,4 days 04:05:00
25,-1,4525,4615,5094.170903,5106.890137,-12.719233,-0.002497,2024-02-29 09:35:00-05:00,2024-03-01 10:35:00-05:00,1 days 01:00:00


## RSI背離交易策略

> 當商品價格創新高或新低時，若RSI的數值沒有同步創新，表示市場價格與RSI很有可能開始疲軟，價格太高或太低時很容易吸引反向的力道，價格非常容易產生拉回或反彈，也就是說當價格超買時應該找時機賣出，當價格超賣時應該找時機買入。

hmm... 還不知道怎麼寫